In [33]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [34]:
# Gather Data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])

In [57]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
NOX_IDX = 3
RM_IDX = 4
DIS_IDX = 5
RAD_IDX = 6
TAX_IDX = 7 
PTRATIO_IDX = 8
B_IDX = 9
LSTAT_IDX = 10

# property_stats = np.ndarray(shape=(1,11))
# property_stats = features['CRIM'].mean()
# property_stats[0][ZN_IDX] = features['ZN'].mean()
# property_stats[0][CHAS_IDX] = features['CHAS'].mean()

property_stats = features.mean().values.reshape(1,11)

In [36]:
type(features.mean())

pandas.core.series.Series

In [37]:
features.mean().values.reshape(1,11)

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [47]:
regr = LinearRegression().fit(features,target)
fitted_vals = regr.predict(features)

# Challenge: calculate the MSE and RMSE using sklearn
MSE = mean_squared_error(target, fitted_vals,squared=True)
RMSE = mean_squared_error(target, fitted_vals,squared=False)

In [98]:
def get_log_estimate(nr_rooms, 
                    students_per_classroom,
                    next_to_river=False,
                    high_confidence=True):
    
    # Configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    
    # Make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calc Range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    return log_estimate, upper_bound, lower_bound, interval

In [99]:
get_log_estimate(3, 20, next_to_river=True, high_confidence=False)

(2.776758191480399, 2.9642703266775294, 2.589246056283269, 68)

In [100]:
np.median(boston_dataset.target)

21.2

In [110]:
# Challenge : Write the python code that converts the log price estimate using 1970s prices
# as well as the upper and the lower bounds to todays's prices. Round the values to the
# nearest 1000 dollars. 

ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(9, students_per_classroom=15,
                                              next_to_river=False, high_confidence=False)

# Conver to today's dollars
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR


# Round the dollar values to nearest thousand
rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_low = np.around(dollar_low, -3)
                        

print(f'The estimated property value is {rounded_est}.')
print(f'At {conf}% confidence the valuation range is.')
print(f'USE {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

The estimated property value is 827000.0.
At 68% confidence the valuation range is.
USE 685000.0 at the lower end to USD 997000.0 at the high end.


In [123]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """Estimate the price of a property in Boston.
    
    Keyword arguments:
    rm -- number of rooms in the property.
    ptratio -- number of students per teacher in the classsroom for the school in the areas.
    chas -- True if the property is next to the river, False otherwise.
    large_range -- True forr a 95% preiction interval, False for a 68% interval.
    
    """
    
    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try again.')
        return
    
    log_est, upper, lower, conf = get_log_estimate(rm,
                                                   students_per_classroom=ptratio,
                                                   next_to_river=chas,
                                                   high_confidence=large_range)

    # Conver to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated property value is {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is.')
    print(f'USE {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

In [124]:
get_dollar_estimate(rm=2, ptratio=1, chas=True)

The estimated property value is 822000.0.
At 95% confidence the valuation range is.
USE 565000.0 at the lower end to USD 1196000.0 at the high end.


In [125]:
import boston_valuation as val
val.get_dollar_estimate(6, 12, True)

The estimated property value is 783000.0.
At 95% confidence the valuation range is.
USE 538000.0 at the lower end to USD 1139000.0 at the high end.
